<a href="https://colab.research.google.com/github/nisitha13/Django_student_management/blob/main/MovieLens%20Ratings%20Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://apache.mirrors.lucidnetworks.net/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark


tar: spark-3.4.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [2]:
!pip -q install pyspark==3.5.1


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MovieLensBigData").getOrCreate()
print("Spark version:", spark.version)
spark


Spark version: 3.5.1


In [6]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q -o ml-100k.zip
!ls -l ml-100k | head



--2025-08-16 06:58:03--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  20.4MB/s    in 0.2s    

2025-08-16 06:58:03 (20.4 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

total 15776
-rwxr-x--- 1 root root     716 Jul 19  2000 allbut.pl
-rwxr-x--- 1 root root     643 Jul 19  2000 mku.sh
-rw-r----- 1 root root    6750 Jan 29  2016 README
-rw-r----- 1 root root 1586544 Mar  8  2001 u1.base
-rw-r----- 1 root root  392629 Mar  8  2001 u1.test
-rw-r----- 1 root root 1583948 Mar  8  2001 u2.base
-rw-r----- 1 root root  395225 Mar  8  2001 u2.test
-rw-r----- 1 root root 1582546 Mar  8  2001 u3.base
-rw-r----- 1 root root  39

In [7]:
from pyspark.sql.types import IntegerType, StructType, StructField

# Schema for ratings dataset
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", IntegerType(), True),
])

# Load the ratings file
ratings = spark.read.csv("ml-100k/u.data", sep="\t", schema=ratings_schema)

# Show first 5 rows
ratings.show(5)

# Show schema
ratings.printSchema()

# Count rows
print("Total ratings:", ratings.count())


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|   196|    242|     3|881250949|
|   186|    302|     3|891717742|
|    22|    377|     1|878887116|
|   244|     51|     2|880606923|
|   166|    346|     1|886397596|
+------+-------+------+---------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)

Total ratings: 100000


In [8]:
from pyspark.sql.functions import col

# Load the raw movies file (pipe-separated)
movies_raw = spark.read.csv("ml-100k/u.item", sep="|", inferSchema=True)

# Select only first 2 columns (movieId, title)
movies = movies_raw.select(
    col("_c0").cast("int").alias("movieId"),
    col("_c1").cast("string").alias("title")
)

# Show first few movies
movies.show(5)
print("Total movies:", movies.count())


+-------+-----------------+
|movieId|            title|
+-------+-----------------+
|      1| Toy Story (1995)|
|      2| GoldenEye (1995)|
|      3|Four Rooms (1995)|
|      4|Get Shorty (1995)|
|      5|   Copycat (1995)|
+-------+-----------------+
only showing top 5 rows

Total movies: 1682


In [9]:
from pyspark.sql.functions import avg, count

movie_stats = (ratings
    .groupBy("movieId")
    .agg(
        avg("rating").alias("avg_rating"),
        count("*").alias("rating_count")
    ))

# Keep only movies with at least 100 ratings (popular ones)
popular_movies = movie_stats.filter(col("rating_count") >= 100)

result_avg = (popular_movies
    .join(movies, on="movieId", how="left")
    .orderBy(col("avg_rating").desc(), col("rating_count").desc()))

result_avg.show(10, truncate=False)


+-------+------------------+------------+--------------------------------+
|movieId|avg_rating        |rating_count|title                           |
+-------+------------------+------------+--------------------------------+
|408    |4.491071428571429 |112         |Close Shave, A (1995)           |
|318    |4.466442953020135 |298         |Schindler's List (1993)         |
|169    |4.466101694915254 |118         |Wrong Trousers, The (1993)      |
|483    |4.45679012345679  |243         |Casablanca (1942)               |
|64     |4.445229681978798 |283         |Shawshank Redemption, The (1994)|
|603    |4.3875598086124405|209         |Rear Window (1954)              |
|12     |4.385767790262173 |267         |Usual Suspects, The (1995)      |
|50     |4.3584905660377355|583         |Star Wars (1977)                |
|178    |4.344             |125         |12 Angry Men (1957)             |
|134    |4.292929292929293 |198         |Citizen Kane (1941)             |
+-------+----------------

In [12]:
popular_movies = movie_stats.filter(col("rating_count") >= 100)


In [13]:
result_avg = popular_movies.join(movies, on="movieId", how="left") \
                           .orderBy(col("avg_rating").desc(), col("rating_count").desc())


In [10]:
most_rated = (movie_stats
    .join(movies, "movieId", "left")
    .orderBy(col("rating_count").desc()))

most_rated.show(10, truncate=False)


+-------+------------------+------------+-----------------------------+
|movieId|avg_rating        |rating_count|title                        |
+-------+------------------+------------+-----------------------------+
|50     |4.3584905660377355|583         |Star Wars (1977)             |
|258    |3.8035363457760316|509         |Contact (1997)               |
|100    |4.155511811023622 |508         |Fargo (1996)                 |
|181    |4.007889546351085 |507         |Return of the Jedi (1983)    |
|294    |3.156701030927835 |485         |Liar Liar (1997)             |
|286    |3.656964656964657 |481         |English Patient, The (1996)  |
|288    |3.4414225941422596|478         |Scream (1996)                |
|1      |3.8783185840707963|452         |Toy Story (1995)             |
|300    |3.6310904872389793|431         |Air Force One (1997)         |
|121    |3.438228438228438 |429         |Independence Day (ID4) (1996)|
+-------+------------------+------------+-----------------------

In [11]:
user_activity = (ratings
    .groupBy("userId")
    .agg(count("*").alias("num_ratings"))
    .orderBy(col("num_ratings").desc()))

user_activity.show(10)


+------+-----------+
|userId|num_ratings|
+------+-----------+
|   405|        737|
|   655|        685|
|    13|        636|
|   450|        540|
|   276|        518|
|   416|        493|
|   537|        490|
|   303|        484|
|   234|        480|
|   393|        448|
+------+-----------+
only showing top 10 rows

